In [1]:
# importing the necessary libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sqlite3 
import os
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import display, HTML
%matplotlib inline

In [2]:
# connecting the database
conn = sqlite3.connect('database.sqlite')
c = conn.cursor()

In [3]:
# printing all the tables present in database.sqlite 
for row in c.execute("SELECT name FROM sqlite_master WHERE type='table';"):
    print(list(row))

['sqlite_sequence']
['Player_Attributes']
['Player']
['Match']
['League']
['Country']
['Team']
['Team_Attributes']


In [4]:
# Read data into DataFrames from all the available tables

PlayerAttribute_df = pd.read_sql("Select * from Player_Attributes",conn)

Player_df = pd.read_sql("Select * from Player",conn)

Match_df = pd.read_sql("Select * from Match",conn)

League_df = pd.read_sql("Select * from League",conn)

Country_df = pd.read_sql("Select * from Country",conn)

Team_df = pd.read_sql("Select * from Team",conn)

TeamAttributes_df = pd.read_sql("Select * from Team_Attributes",conn)

In [5]:
#Function to find the proportion of missing values

def missing_values(input_df,null_percent=0):

    output_df = pd.DataFrame({'missing_count':[],'missing_prop':[]})
    nullcount_df = input_df.isna().sum()
    output_df['missing_count'] = nullcount_df.iloc[0:]
    output_df['missing_prop'] = output_df['missing_count']/len(input_df.index)*100
    output_df.index=nullcount_df.index
    if null_percent>0:
        return output_df[output_df['missing_prop']>=null_percent]
    else:
        return output_df

In [7]:
missing_values(Country_df,null_percent=0)
missing_values(PlayerAttribute_df,null_percent=0)
missing_values(Player_df,null_percent=0)
missing_values(Match_df,null_percent=0)
missing_values(League_df,null_percent=0)
missing_values(Team_df,null_percent=0)
missing_values(TeamAttributes_df,null_percent=0)

,missing_count,missing_prop
id,0,0.000000
team_fifa_api_id,0,0.000000
team_api_id,0,0.000000
date,0,0.000000
buildUpPlaySpeed,0,0.000000
buildUpPlaySpeedClass,0,0.000000
buildUpPlayDribbling,969,66.460905
buildUpPlayDribblingClass,0,0.000000
buildUpPlayPassing,0,0.000000
buildUpPlayPassingClass,0,0.000000


In [8]:
# Number teams each country have
query = pd.read_sql_query(
    '''
        SELECT 
              c.name AS Country,
              COUNT(DISTINCT(team_long_name)) AS 'No. of Teams'
              FROM Match AS m
              LEFT JOIN Country AS c
              ON m.country_id = c.id
              LEFT JOIN Team AS t 
              ON m.home_team_api_id = t.team_api_id
              GROUP BY Country
    ''', conn
)
query

,Country,No. of Teams
0,Belgium,24
1,England,34
2,France,35
3,Germany,30
4,Italy,32
5,Netherlands,25
6,Poland,22
7,Portugal,29
8,Scotland,17
9,Spain,33


In [9]:
# total goals scored by home team grouped by country and Season
query1 = pd.read_sql_query(
    '''
        SELECT c.name AS Country,
               m.season AS Season,
               SUM(m.home_team_goal) AS 'Home Goal',
               SUM(m.away_team_goal) AS 'Away Goal'
        FROM Match as m 
        LEFT JOIN country AS c
        ON m.country_id = c.id
        GROUP BY Country, Season    
        ORDER BY Country     
    ''', conn
)
query1

,Country,Season,Home Goal,Away Goal
0,Belgium,2008/2009,499,356
1,Belgium,2009/2010,308,257
2,Belgium,2010/2011,382,253
3,Belgium,2011/2012,421,270
4,Belgium,2012/2013,375,328
...,...,...,...,...
83,Switzerland,2011/2012,235,190
84,Switzerland,2012/2013,272,190
85,Switzerland,2013/2014,290,230
86,Switzerland,2014/2015,289,228


In [10]:
# goals scored by each team grouped by season
query2 = pd.read_sql_query(
    '''
        SELECT t.team_long_name AS Team, 
               m.season as Season,
               SUM(m.home_team_goal) AS 'Home Goal',
               SUM(m.away_team_goal) AS 'Away Goal'
        FROM Match AS m
        LEFT JOIN Team AS t
        ON m.home_team_api_id = t.team_api_id 
        GROUP BY Team, Season
        ORDER BY Team
    ''', conn
)
query2

,Team,Season,Home Goal,Away Goal
0,1. FC Kaiserslautern,2010/2011,25,19
1,1. FC Kaiserslautern,2011/2012,12,28
2,1. FC Köln,2008/2009,14,25
3,1. FC Köln,2009/2010,18,29
4,1. FC Köln,2010/2011,30,21
...,...,...,...,...
1473,Śląsk Wrocław,2011/2012,26,14
1474,Śląsk Wrocław,2012/2013,23,15
1475,Śląsk Wrocław,2013/2014,21,16
1476,Śląsk Wrocław,2014/2015,21,9


In [11]:
query3 = pd.read_sql_query(
    '''
        SELECT c.name AS Country,
               t.team_long_name AS Team,
               m.season AS Season,
               SUM(m.home_team_goal) AS 'Home Goal',
               SUM(m.away_team_goal) AS 'Away Goal'
        FROM Match as m
        LEFT JOIN Country AS c
        ON m.country_id = c.id
        LEFT JOIN Team AS t
        ON m.home_team_api_id = t.team_api_id
        GROUP BY Country, Team, Season
        ORDER BY Country
    ''', conn
)
query3

,Country,Team,Season,Home Goal,Away Goal
0,Belgium,Beerschot AC,2008/2009,32,19
1,Belgium,Beerschot AC,2009/2010,20,22
2,Belgium,Beerschot AC,2010/2011,14,10
3,Belgium,Beerschot AC,2011/2012,26,13
4,Belgium,Beerschot AC,2012/2013,20,34
...,...,...,...,...,...
1473,Switzerland,Neuchâtel Xamax,2009/2010,35,27
1474,Switzerland,Neuchâtel Xamax,2010/2011,23,35
1475,Switzerland,Neuchâtel Xamax,2011/2012,12,10
1476,Switzerland,Servette FC,2011/2012,23,27


In [12]:
# count of matches won, lost and tie
query4 = pd.read_sql_query(
    '''
    WITH sub_q AS (
        SELECT     
                   c.name AS Country,
                   t.team_long_name AS Team,
                   COUNT(CASE WHEN m.home_team_goal > away_team_goal THEN 'Win' END) AS Won,
                   COUNT(CASE WHEN m.home_team_goal < away_team_goal THEN 'Lost' END) AS Lost,
                   COUNT(CASE WHEN m.home_team_goal = away_team_goal THEN 'Draw' END) AS Draw
         FROM Match AS m
         LEFT JOIN Country AS c
         ON m.country_id = c.id
         LEFT JOIN Team as t
         ON m.home_team_api_id = t.team_api_id
         GROUP BY Country, Team
         ORDER BY Country
         )
         SELECT ROW_NUMBER() OVER(ORDER BY Won DESC) AS 'Row Number',
                Country, 
                Team, 
                Won, 
                Lost,
                Draw
        FROM sub_q
    ''', conn, index_col='Row Number'
    )
query4

,Country,Team,Won,Lost,Draw
Row Number,,,,,
1,Spain,FC Barcelona,131,9,12
2,Spain,Real Madrid CF,129,12,11
3,Scotland,Celtic,120,9,23
4,England,Manchester United,116,20,16
5,England,Manchester City,113,21,18
...,...,...,...,...,...
292,Germany,DSC Arminia Bielefeld,2,7,8
293,Germany,SV Darmstadt 98,2,9,6
294,Scotland,Dunfermline Athletic,1,11,7


In [13]:
# total home goals scored grouped by country and team
query5 = pd.read_sql_query(
    '''
        SELECT name AS Name,
            team_long_name AS Team,
            --STRFTIME('%Y', date) AS Year, 
            SUM(home_team_goal) AS Goal
        FROM Match AS m
        LEFT JOIN Country as c
        ON m.country_id = c.id
        LEFT JOIN Team AS t
        ON m.home_team_api_id = t.team_api_id  
        GROUP BY Name, Team
        ORDER BY Goal DESC
        
    ''', conn
)
query5

,Name,Team,Goal
0,Spain,Real Madrid CF,505
1,Spain,FC Barcelona,495
2,Scotland,Celtic,389
3,Germany,FC Bayern Munich,382
4,Netherlands,PSV,370
...,...,...,...
291,France,AC Arles-Avignon,14
292,Portugal,Trofense,14
293,Portugal,Feirense,13
294,Spain,Córdoba CF,12


In [14]:
# extracting Year, Month, Week and day from date
query7 = pd.read_sql_query(
    '''
        SELECT date AS Date,
               STRFTIME('%Y', date) AS Year,
               STRFTIME('%m', date) AS Month,
               STRFTIME('%w', date) AS Week,
               STRFTIME('%d', date) AS Day
        FROM Match
    ''', conn
)
query7

,Date,Year,Month,Week,Day
0,2008-08-17 00:00:00,2008,08,0,17
1,2008-08-16 00:00:00,2008,08,6,16
2,2008-08-16 00:00:00,2008,08,6,16
3,2008-08-17 00:00:00,2008,08,0,17
4,2008-08-16 00:00:00,2008,08,6,16
...,...,...,...,...,...
25974,2015-09-22 00:00:00,2015,09,2,22
25975,2015-09-23 00:00:00,2015,09,3,23
25976,2015-09-23 00:00:00,2015,09,3,23
25977,2015-09-22 00:00:00,2015,09,2,22


In [15]:
# count of matches won, lost and tie
query8 = pd.read_sql_query(
    '''
    WITH sub_q AS (
        SELECT     
                   c.name AS Country,
                   season AS Season,
                   t.team_long_name AS Team,
                   --m.home_team_goal AS home_goal,
                   --m.away_team_goal AS away_goal,
                   COUNT(CASE WHEN m.home_team_goal > away_team_goal THEN 'Win' END) AS Won,
                   COUNT(CASE WHEN m.home_team_goal < away_team_goal THEN 'Lost' END) AS Lost,
                   COUNT(CASE WHEN m.home_team_goal = away_team_goal THEN 'Draw' END) AS Draw
         FROM Match AS m
         LEFT JOIN Country AS c
         ON m.country_id = c.id
         LEFT JOIN Team as t
         ON m.home_team_api_id = t.team_api_id
         GROUP BY Country, Season
         ORDER BY Country
         )
         SELECT ROW_NUMBER() OVER(ORDER BY Won DESC) AS 'Row Number',
                Country, 
                Season,
                Won, 
                Lost,
                Draw
        FROM sub_q
    ''', conn, index_col='Row Number'
    )
query8

,Country,Season,Won,Lost,Draw
Row Number,,,,,
1,Spain,2010/2011,197,104,79
2,Spain,2009/2010,194,91,95
3,England,2009/2010,193,91,96
4,Italy,2008/2009,192,93,95
5,Spain,2012/2013,189,107,84
...,...,...,...,...,...
84,Switzerland,2015/2016,80,53,47
85,Switzerland,2014/2015,76,56,48
86,Switzerland,2010/2011,72,57,51
